# ETL
## Exploración no estadística, y disponibilización de los datos del archivo Homicidios.xlsx

In [2]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

### Extracción
Importo los datos contenidos en el archivo homicidios.xlsx, que trae dos hojas: HECHOS y VICTIMAS. Almaceno en dos dataframes independientes

In [ ]:
df_HH = pd.read_excel('homicidios.xlsx', sheet_name= "HECHOS")
df_HV = pd.read_excel('homicidios.xlsx', sheet_name= "VICTIMAS")

### Análisis exploratorio inicial y transformación de datos para dataframe de HECHOS (df_HH)

In [ ]:
df_HH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

#### Revisión de Datos Nulos

In [ ]:
df_HH.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

De esta primera exploración, sumada a una revisión manual al abrir el archivo en excel, se puede apreciar que es un dataset bastante limpio y casi listo para trabajar. Tiene información completa, es decir sin datos nulos, en las columnas ID, N_VICITMAS, FECHA, AAA, MM, DD, HORA, LUGAR DEL HECHO, TIPO DE CALLE, COMUNA, PARTICIPANTES, VICTIMA, ACUSADO.

EN LAS COLUMNAS XY (CABA), pos x, pos y, aparentemente no hay datos nulos, pero al revisar en excel se pueden encontrar varios registros en los cuales no hay realmente información en estos campos sino puntos (.), lo que para efectos prácticos equivale a valores nulos o vacíos.

Otras columnas con algunos datos nulos son las que corresponden al desglose de la dirección: Calle, Altura, Cruce, Dirección Normalizada.

Los nombres que traen las columnas tal vez sugerían ya esa información, pues las columnas que están completas tienen su nombre todo escrito en mayúsculas (Con excepción de XY CABA), y las incompletas están en minúsculas.

Un reto grande con el que me encuentro, al no ser ciudadana Argentina, es entender cómo funcionan las direcciones en la ciudad.

#### Columnas XY (CABA), pos x, pos y

La geolocalización es importante para los ánalisis que se deben realizar, así que no quiero eliminar los registros en los que estos datos no existen. Intentaré utilizar la información que poseo sobre el lugar del accidente para completar la que hace falta. Para esta tarea utilizaré GeoPy que es una librería de python para acceder a servicios de geocodificación.

Mi objetivo es conseguir los valores de las coordenadas X, Y. El valor de XY(CABA) -geocodificación plana-, podría ser útil a la hora de utilizar mapas pero eso lo decidiré más adelante.

In [ ]:
#Observo los registros que necesito completar:
df_sinXY = df_HH[df_HH['XY (CABA)']=='Point (. .)'][['ID','LUGAR_DEL_HECHO', 'Dirección Normalizada', 'XY (CABA)', 'pos x', 'pos y']]
df_sinXY.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 35 to 621
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     14 non-null     object
 1   LUGAR_DEL_HECHO        14 non-null     object
 2   Dirección Normalizada  6 non-null      object
 3   XY (CABA)              14 non-null     object
 4   pos x                  14 non-null     object
 5   pos y                  14 non-null     object
dtypes: object(6)
memory usage: 784.0+ bytes


In [ ]:
df_sinXY

,ID,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y
35,2016-0049,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238
38,2016-0052,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.
71,2016-0096,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973
106,2016-0136,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.
119,2016-0151,SD,NaN,Point (. .),.,.
139,2016-0174,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.
176,2017-0042,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.
180,2017-0050,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.
181,2017-0051,AU DELLEPIANE 2400,NaN,Point (. .),.,.
256,2017-0140,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.


Son 12 registros que no tienen valores en pos x y pos y. Y 14 que no tienen XY (CABA). Utilizo geolocator para conseguir las coordenadas geográficas a partir del LUGAR_DEL_HECHO

In [ ]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=10)

def do_geocode(address):
     return geolocator.geocode(address)

In [ ]:
df_sinXY['coordenadas'] =df_sinXY['LUGAR_DEL_HECHO'].apply(do_geocode).apply(lambda loc: tuple(loc.point) if loc else None)
df_sinXY

,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y,coordenadas
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238,None
38,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.,None
71,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973,None
106,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.,None
119,SD,NaN,Point (. .),.,.,"(14.5844444, 29.4917691, 0.0)"
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.,"(-34.6425804, -58.461474, 0.0)"
176,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.,None
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.,None
181,AU DELLEPIANE 2400,NaN,Point (. .),.,.,"(-31.4271733, -64.2087771, 0.0)"
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.,None


Sé que esto es código redundante, pero siendo tan pocos los datos nuevos que debo añadir al dataframe, decido hacerlo de forma manual. Ya conozco cuáles registros lograron obtener sus coordenadas de latitud y longitud, entonces realizo de nuevo el proceso, uno a uno, y reemplazo los datos en sus columnas correspondientes. Los índices de los registros modificados son 559, 181 y 139.

In [ ]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=50)
location = geolocator.geocode("MURGUIONDO, Buenos Aires, Argentina")
df_HH['pos y'][559] = location.latitude
df_HH['pos x'][559] = location.longitude

Lleno con ceros el resto de los datos que quedan vacíos en XY (CABA), pos x y pos y, para efectos del manejo en dashboard

In [ ]:
df_HH['pos x'] = df_HH['pos x'].replace('.', 0)
df_HH['pos y'] = df_HH['pos y'].replace('.', 0)
df_HH['XY (CABA)'] = df_HH['XY (CABA)'].replace('Point (. .)', 0)

#### Columnas 'Altura' y 'Cruce'

Son las dos columnas que presentan más valores nulos, y puedo ver que son complementarias en la información que contienen.
Indican si el accidente ocurrió en una esquina (cruce de vías), o en la mitad de una cuadra. Es una información importante que se puede resumir en una sola columna, en la que consignaré si el accidente ocurrió o no ocurrió en un cruce. Esto implicará conservar la columna 'Cruce' cambiando sus valores a True/False y eliminar la columna 'Altura'.
De esta manera también la Columna 'Cruce' quedaría completa, sin valores nulos.

In [ ]:
df_HH = df_HH.drop('Altura', axis=1)

In [ ]:
df_HH['Cruce'] = df_HH['Cruce'].notna()
df_HH.head(15)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,27 DE FEBRERO AV.,True,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
6,2016-0009,1,2016-01-24,2016,1,24,19:10:00,19,NOGOYA Y JOAQUIN V. GONZALES,CALLE,NOGOYA,True,"NOGOYA y GONZALEZ, JOAQUIN V.",11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,MOTO-AUTO,MOTO,AUTO
7,2016-0010,1,2016-01-29,2016,1,29,15:20:00,15,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,MOTO-AUTO,MOTO,AUTO
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,BELGRANO AV.,True,"BELGRANO AV. e IRIGOYEN, BERNARDO DE",1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,MOTO-CARGAS,MOTO,CARGAS
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO


#### Columna ID y revisión de duplicados

No necesita ningún cambio, únicamente validación del tipo de dato que debe ser string.
Uso esta columna para verificar si hay duplicados. La revisión me muestra que no los hay.

In [ ]:
df_HH['ID'].duplicated().sum()

0

#### Columna N_VICTIMAS
Esta columna contiene valores de tipo entero y no tiene valores nulos. Está lista para ser utilizada.

#### Columnas FECHA, AAAA, MM, DD, HORA Y HH

Estas columnas contienen información que puede ser redundante, pero no me estorba, no tienen valores nulos que puedan entorpecer las consultas y por lo tanto decido conservarlas como están. Sólo falta verificar tipos de datos.

In [ ]:
#Detecto que hay al menos un registro con SD en la columna HORA. Reviso cuál o cuáles son
df_HH[df_HH['HORA']=='SD'][['ID','FECHA','HORA', 'HH', ]]


,ID,FECHA,HORA,HH
518,2019-0103,2019-12-18,SD,SD


In [ ]:
#Cambio el valor SD por 0 para poder convertir luego los tipos de datos de estas columnas
df_HH['HORA'][518]=0
df_HH['HH'][518]=0

#### Columnas LUGAR_DEL_HECHO y Direccion Normalizada
Ambas columnas almacenan la misma información, sólo cambia el formato. En 'Dirección Normalizada' Hay algunos valores nulos.

In [ ]:
df_HH[df_HH['Dirección Normalizada'].isnull()][['Dirección Normalizada', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE']]

,Dirección Normalizada,LUGAR_DEL_HECHO,TIPO_DE_CALLE
38,NaN,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,NaN,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,NaN,SD,CALLE
180,NaN,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,NaN,AU DELLEPIANE 2400,AUTOPISTA
313,NaN,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,NaN,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,NaN,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


Son pocos datos pero no tengo información para construir esa dirección con el formato requerido, así que le asigno el valor SD para conservar estos registros

In [ ]:
# Se reemplazan los valores nulos con SD
df_HH['Dirección Normalizada'].fillna("SD", inplace=True)
# Se verifican los datos imputados
df_HH[df_HH['Dirección Normalizada']== 'SD'][['Dirección Normalizada', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE']]

,Dirección Normalizada,LUGAR_DEL_HECHO,TIPO_DE_CALLE
38,SD,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,SD,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,SD,SD,CALLE
180,SD,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,SD,AU DELLEPIANE 2400,AUTOPISTA
313,SD,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,SD,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,SD,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


#### Columna Calle
Esta columna tiene un sólo valor nulo. Reviso si puedo corregirlo manualmente con la información de los otros campos

In [ ]:
df_HH[df_HH['Calle'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,NaN,False,SD,0,0,0,0,PEATON-SD,PEATON,SD


Es un campo que contiene muy poca información. Nada sobre el lugar de accidente ni sobre el acusado. Considero que todos los registros suman en las estadísticas de temas tan delicados como este, así que decido conservar el registro, colocando SD en los campos que está vacíos.

In [ ]:
df_HH['Calle'][df_HH['ID']=='2016-0151'] = 'SD'

Hago un nuevo conteo de valores nulos para verificar que no haya quedado ninguno

In [ ]:
df_HH.isnull().sum()

ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Cruce                    0
Dirección Normalizada    0
COMUNA                   0
XY (CABA)                0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64

#### Columnas TIPO_DE_CALLE, COMUNA, PARTICIPANTES

Estas columnas no tienen valores nulos. Están listas para ser utilizadas.

#### Columnas VICTIMA y ACUSADO
Reviso si las categorías presentes en estas columnas corresponden a las informadas en el Diccionario de Datos

In [ ]:
df_HH['ACUSADO'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

Encuentro un valor que no corresponde al los del diccionario, 'MULTIPLE', voy a revisar los registros que lo contienen.

In [ ]:
df_HH[df_HH['ACUSADO'].isin(['MULTIPLE'])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
31,2016-0044,1,2016-03-31,2016,3,31,19:40:00,19,AV JOSE MARIA MORENO Y BALBASTRO,AVENIDA,"MORENO, JOSE MARIA AV.",True,"MORENO, JOSE MARIA AV. y BALBASTRO",7,Point (102857.73817324 99148.76714504),-58.43213235,-34.63693953,MULTIPLE,MOTO,MULTIPLE
143,2016-0178,1,2016-12-30,2016,12,30,16:18:00,16,REPUBLICA BOLIVARIANA DE VENEZUELA Y SAAVEDRA,CALLE,REPUBLICA BOLIVARIANA DE VENEZUELA,True,REPUBLICA BOLIVARIANA DE VENEZUELA y SAAVEDRA,3,Point (105541.07964363 101532.48653010),-58.40288226,-34.61544091,MULTIPLE,PEATON,MULTIPLE
146,2017-0007,1,2017-01-11,2017,1,11,17:00:00,17,LIMA Y MEXICO,CALLE,LIMA,True,LIMA y MEXICO,1,Point (107491.43483173 101547.26343171),-58.38161660,-34.61529543,MULTIPLE,PEATON,MULTIPLE
153,2017-0014,1,2017-01-28,2017,1,28,08:27:34,8,AV. DONADO Y ARIAS,AVENIDA,DONADO,True,DONADO y ARIAS,12,Point (97378.59173196 108741.41437861),-58.49185972,-34.55046754,MULTIPLE,MOTO,MULTIPLE
174,2017-0040,1,2017-04-03,2017,4,3,09:40:00,9,CAMINO CIUDAD UNUVERSITARIA Y AV. TTE. GUIRALDES,AVENIDA,"GUIRALDES, INT.",False,"GUIRALDES, INT. 2699",13,Point (102290.82648731 109281.08885461),-58.43834197,-34.54560369,MULTIPLE,AUTO,MULTIPLE
225,2017-0101,1,2017-08-23,2017,8,23,10:00:00,10,AV. FCO. LACROZE Y ZAPATA,AVENIDA,"LACROZE, FEDERICO AV.",True,"LACROZE, FEDERICO AV. y ZAPATA",13,Point (101644.77699748 106480.78805418),-58.44537508,-34.57084818,MULTIPLE,PEATON,MULTIPLE
277,2018-0003,1,2018-01-05,2018,1,5,10:12:00,10,"Pueyrredon, Honorio, Dr. Av. y Galicia",AVENIDA,"PUEYRREDON, HONORIO, DR. AV.",True,"PUEYRREDON, HONORIO, DR. AV. y GALICIA",15,Point (101348.13972450 102808.02053976),-58.44860217,-34.60395657,MULTIPLE,PEATON,MULTIPLE
278,2018-0004,1,2018-01-09,2018,1,9,09:40:00,9,Del Libertador Av. y Manzanares,AVENIDA,DEL LIBERTADOR AV.,True,DEL LIBERTADOR AV. y MANZANARES,13,Point (100183.96103971 109533.84269558),-58.46129513,-34.54332766,MULTIPLE,MOTO,MULTIPLE
355,2018-0081,1,2018-07-29,2018,7,29,06:30:00,6,Castañares Av. y Rivera Indarte Av.,AVENIDA,CASTANARES AV.,True,CASTAÃ‘ARES AV. y RIVERA INDARTE AV.,7,Point (101453.89687005 98180.39027169),-58.44744156,-34.64567171,MULTIPLE,PEATON,MULTIPLE
375,2018-0101,1,2018-09-21,2018,9,21,00:15:00,0,"Paz, Gral. Av. y Zapiola",GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y ZAPIOLA",12,Point (98376.44189669 109821.71138325),-58.48098594,-34.54073130,MULTIPLE,AUTO,MULTIPLE


Veo que en todos estos accidentes los participantes fueron múltiples y hubo una sola víctima fatal. Por ahora dejo esto como está, sin realizar ninguna modificación.

In [ ]:
df_HH['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

Encuentro dos valores que no corresponden, OBJETO FIJO y PEATON_MOTO, entonces voy a revisar los registros que los contienen

In [ ]:
df_HH[df_HH['VICTIMA'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


La información es confusa, pero revisando la hoja de Vícitimas para estos dos casos, se puede ver que en ambos accidentes hubo dos víctimas fatales. 
En el primer caso no tiene mucho sentido que la víctima haya sido un OBJETO FIJO, es cierto que el accidente fue una colisión contra un objeto fijo pero las víctimas fueron PASAJERO_ACOMPAÑANTE.
En el segundo caso, las vícitmas fatales fueron un PEATON y un PASAJERO_ACOMPAÑANTE.

Voy a revisar otros casos que hayan tenido 2 víctimas, para ver cómo están registrados en esta columna.

In [ ]:
df_HH[df_HH['N_VICTIMAS']==2]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
29,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,DIRECTORIO AV.,True,DIRECTORIO AV. y RIVERA INDARTE AV.,7,Point (100232.38564985 99530.25843190),-58.46076550,-34.63350444,MOTO-CARGAS,MOTO,CARGAS
97,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,IRIGOYEN,True,IRIGOYEN y TINOGASTA,10,Point (94275.54271123 100886.87954649),-58.52572109,-34.62125906,AUTO-CARGAS,AUTO,CARGAS
161,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,"MORENO, PERITO AV.",True,"MORENO, PERITO AV. y FOURNIER",4,Point (104113.67806500 97722.68219304),-58.41842777,-34.64979057,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
171,2017-0036,2,2017-03-29,2017,3,29,18:00:00,18,CURUPAYTI Y COLECTORA AV. GRAL. PAZ,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y CAMPANA",12,Point (95300.18060161 105287.23850698),-58.51452347,-34.58159762,MOTO-PASAJEROS,MOTO,PASAJEROS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,0,0,0,MOTO-CARGAS,MOTO,CARGAS
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
234,2017-0112,2,2017-09-10,2017,9,10,09:32:17,9,AU 25 DE MAYO Y AV. VARELA,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO y VARELA AV.,7,Point (100479.14412395 98812.52258620),-58.45807401,-34.63997430,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
244,2017-0126,2,2017-10-14,2017,10,14,06:15:00,6,AV. RIESTRA Y CAÑADA DE GOMEZ,AVENIDA,RIESTRA AV.,True,RIESTRA AV. y CAÃ‘ADA DE GOMEZ,8,Point (98466.20276785 94207.42419155),-58.48003734,-34.68148508,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
267,2017-0153,2,2017-12-11,2017,12,11,02:00:00,2,ROSARIO Y AV. LA PLATA,AVENIDA,ROSARIO,True,ROSARIO y LA PLATA AV.,6,Point (103186.65733155 101217.09689755),-58.42855267,-34.61829385,MOTO-PASAJEROS,MOTO,PASAJEROS
289,2018-0015,2,2018-02-06,2018,2,6,01:00:00,1,Australia Av. 2800,AVENIDA,AUSTRALIA AV.,False,AUSTRALIA AV. 2800,4,Point (107053.47849253 98047.93724144),-58.38636298,-34.64684275,MOTO-PASAJEROS,MOTO,PASAJEROS


Encuentro que no hay información sobre dos víctimas en la columna VICTIMA. Por ahora dejaré esto sin realizar modificación y lo atenderé de nuevo luego de realizar el análisis al dataframe df_HV, correspondiente a la hoja VICTIMAS  extraída del documento de excel.

### Verificacion de tipos de variables

In [ ]:
df_HH.dtypes

ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Cruce                              bool
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                            object
pos y                            object
PARTICIPANTES                    object
VICTIMA                          object
ACUSADO                          object
dtype: object

In [ ]:
df_HH['ID'] = df_HH['ID'].astype(str)
df_HH['HORA'] = df_HH['HORA'].astype(str)
df_HH['HH'] = df_HH['HH']. astype(int)
df_HH['LUGAR_DEL_HECHO'] = df_HH['LUGAR_DEL_HECHO']. astype(str)
df_HH['TIPO_DE_CALLE'] = df_HH['TIPO_DE_CALLE']. astype(str)
df_HH['Calle'] = df_HH['Calle']. astype(str)
df_HH['Dirección Normalizada'] = df_HH['Dirección Normalizada']. astype(str)
df_HH['pos x'] = df_HH['pos x']. astype(float)
df_HH['pos y'] = df_HH['pos y']. astype(float)
df_HH['PARTICIPANTES'] = df_HH['PARTICIPANTES']. astype(str)
df_HH['VICTIMA'] = df_HH['VICTIMA']. astype(str)
df_HH['ACUSADO'] = df_HH['ACUSADO']. astype(str)

### Cambios en los nombres de las columnas

In [ ]:

df_HH = df_HH.rename(columns={'N_VICTIMAS': 'Cantidad víctimas',
                                'FECHA': 'Fecha',
                                'AAAA':'Año',
                                'MM':'Mes',
                                'DD':'Día',
                                'HORA': 'Hora',
                                'HH':'Hora entera',
                                'LUGAR_DEL_HECHO': 'Lugar del hecho',
                                'TIPO_DE_CALLE': 'Tipo de Calle',
                                'COMUNA': 'Comuna',
                                'pos x': 'Pos x',
                                'pos y': 'Pos y',
                                'PARTICIPANTES': 'Participantes',                      
                                'VICTIMA': 'Víctima',
                                'ACUSADO': 'Acusado'
                                })

In [ ]:
df_HH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   Cantidad víctimas      696 non-null    int64         
 2   Fecha                  696 non-null    datetime64[ns]
 3   Año                    696 non-null    int64         
 4   Mes                    696 non-null    int64         
 5   Día                    696 non-null    int64         
 6   Hora                   696 non-null    object        
 7   Hora entera            696 non-null    int64         
 8   Lugar del hecho        696 non-null    object        
 9   Tipo de Calle          696 non-null    object        
 10  Calle                  696 non-null    object        
 11  Cruce                  696 non-null    bool          
 12  Dirección Normalizada  696 non-null    object        
 13  Comun

In [ ]:
#Exporto a CSV para poder continuar el trabajo luego
df_HH.to_csv('Homicidios_Hechos.csv')

### Análisis exploratorio inicial y transformación de datos para dataframe de VICTIMAS (df_HV)

In [ ]:
df_HV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [ ]:
df_HV.head(10)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30,2016-01-24 00:00:00
6,2016-0009,2016-01-24,2016,1,24,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,29,2016-01-26 00:00:00
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18,2016-01-29 00:00:00
8,2016-0012,2016-02-08,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22,2016-02-08 00:00:00
9,2016-0013,2016-02-10,2016,2,10,PEATON,PEATON,MASCULINO,16,2016-02-10 00:00:00


Analizando ambos dataframes, observo que los datos de ID, fecha y hora coinciden entre ambas hojas

In [ ]:
df_HH.head(10)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,...,NaN,ESCALADA AV.,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
6,2016-0009,1,2016-01-24,2016,1,24,19:10:00,19,NOGOYA Y JOAQUIN V. GONZALES,CALLE,...,NaN,"GONZALEZ, JOAQUIN V.","NOGOYA y GONZALEZ, JOAQUIN V.",11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,MOTO-AUTO,MOTO,AUTO
7,2016-0010,1,2016-01-29,2016,1,29,15:20:00,15,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,MOTO-AUTO,MOTO,AUTO
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,...,NaN,"IRIGOYEN, BERNARDO DE","BELGRANO AV. e IRIGOYEN, BERNARDO DE",1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,MOTO-CARGAS,MOTO,CARGAS
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,...,1366.0,NaN,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO


También observo que mientras el dataset de Hechos tiene 696 registros, el de vícitmas tiene 717. Esto se debe a que algunos accidentes tienen más de una víctima.

#### Revisión de datos nulos

In [ ]:
df_HV.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

#### Revisión de duplicados

A diferencia del dataframe anterior, no utilizo el ID para verificar duplicados, pues sé que habrá Id's duplicados en los casos en que un accidente haya tenido varias víctimas

In [ ]:
df_HV[df_HV.duplicated()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Al navegar la hoja de cálculo puede ver en diferentes columnas el valor SD, que significa Sin Datos. Voy a revisar esto columna por columna para ver si hay algo que pueda hacer para imputar estos espacios.

In [ ]:
df_HV[df_HV['ID_hecho']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [ ]:
df_HV[df_HV['FECHA']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [ ]:
df_HV[df_HV['AAAA']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [ ]:
df_HV[df_HV['MM']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [ ]:
df_HV[df_HV['DD']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Las columnas correspondientes al ID y las relacionadas con la fecha del accidente están completas.

In [ ]:
df_HV[df_HV['FECHA_FALLECIMIENTO']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
16,2016-0022,2016-02-21,2016,2,21,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,41,SD
19,2016-0027,2016-02-28,2016,2,28,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,34,SD
22,2016-0031,2016-03-08,2016,3,8,CONDUCTOR,MOTO,MASCULINO,21,SD
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
...,...,...,...,...,...,...,...,...,...,...
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,SD
242,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,1,SD
246,2017-0115,2017-09-19,2017,9,19,CONDUCTOR,MOTO,MASCULINO,34,SD
253,2017-0126,2017-10-14,2017,10,14,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,39,SD


Hay 68 registros sin fecha de fallecimiento. Tengo varias ocpiones:
- Rellenar este valor con la fecha del accidente. Observo que el formato de fecha es diferente en ambas columnas, así que tendría que unificar este formato, eso implicaría una revisión y transformación de todos los datos de una de las columnas, uno a uno.
- Eliminar la columna, ya que no me aporta mucho al análisis estadísitico ni a los KPI's. Sería interesante para revisar temas de rapidez de la atención médica, llegada de asistencia médica a la escena, disponibilidad de ambulancias, etc., ya que tal vez, si las víctimas no murieron en el lugar del accidente, una atención médica eficiente y oportuna podría ser determinante para salvar sus vidas; pero no es el área de enfoque de este proyecto.

Me decanto por la opción de eliminar la columna.

In [ ]:
df_HV = df_HV.drop(columns = ['FECHA_FALLECIMIENTO'])

In [ ]:
df_HV[df_HV['ROL']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD
77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD
167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34
208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23
280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77


In [ ]:
df_HV[df_HV['VICTIMA']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD
77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD
167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34
208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23
280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77


In [ ]:
df_HV[df_HV['SEXO']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD
108,2016-0136,2016-10-25,2016,10,25,CONDUCTOR,MOTO,SD,SD
121,2016-0151,2016-11-18,2016,11,18,PEATON,PEATON,SD,SD
138,2016-0171,2016-12-25,2016,12,25,CONDUCTOR,MOTO,SD,SD
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD


En estas tres columnas la proporción de SD es pequeña: en ROL son 11 registros (1.5%), en VICTIMA son 9 registro (1.3%) y en SEXO son 6 registros (0.8%).

Para que no queden vacíos, puedo cambiar los datos faltantes de SEXO y ROL por el valor más frecuente de cada columna, con la tranquilidad de que, al ser tan pocos datos, no voy a alterar significativamente los resultados de los análisis estadísticos.

No voy a hacer ningún cambio en la columna VICTIMA, pues podría generar inconsistencia con los datos sobre la víctima que se encuentran en el dataframe de Hechos. Al hacer merge de ambos dataframes volveré a revisar este campo.

In [ ]:
valor_mas_frecuente = df_HV['SEXO'].mode().iloc[0]
print(f'El valor mas frecuente para SEXO es: {valor_mas_frecuente}')

El valor mas frecuente para SEXO es: MASCULINO


In [ ]:
df_HV['SEXO'][df_HV['SEXO']=='SD'] = 'MASCULINO'

In [ ]:
df_HV[df_HV['SEXO']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD


In [ ]:
valor_mas_frecuente = df_HV['ROL'].mode().iloc[0]
print(f'El valor mas frecuente para ROL es: {valor_mas_frecuente}')

El valor mas frecuente para ROL es: CONDUCTOR


In [ ]:
df_HV['ROL'][df_HV['ROL']=='SD'] = 'CONDUCTOR'

In [ ]:
df_HV[df_HV['ROL']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD


In [ ]:
df_HV[df_HV['EDAD']=='SD'].shape[0]

53

En la columna EDAD el número de registros sin datos es mucho mayor, aunque sigue siendo un porcentaje pequeño: son 45 registros (6.3%). Ya que dejar estos valores como SD (str) me va a generar problemas con el tipo de datos de la columna y asignarles 0 puede afectar los cálculos estadísticos, voy a tomar el promedio de edad según el sexo de las víctimas para rellenar estos faltantes.

In [ ]:
# Reemplazo SD por Na para poder calcular el promedio
df_HV['EDAD'] = df_HV['EDAD'].replace('SD', pd.NA)

promedio_por_genero = df_HV.groupby('SEXO')['EDAD'].mean()
print(f'La edad promedio de FEMENINO es {round(promedio_por_genero["FEMENINO"])} y de MASCULINO es {round(promedio_por_genero["MASCULINO"])}')

# Se llenan los valores NaN en la columna 'edad' utilizando el promedio correspondiente al género
df_HV['EDAD'] = df_HV.apply(lambda row: promedio_por_genero[row['SEXO']] if pd.isna(row['EDAD']) else row['EDAD'], axis=1)
# Convierto de una vez el tipo de dato de esta columna a entero
df_HV['EDAD'] = df_HV['EDAD'].astype(int)

La edad promedio de FEMENINO es 51 y de MASCULINO es 40


In [ ]:
df_HV[df_HV['EDAD']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD


In [ ]:
df_HV.head(20)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30
6,2016-0009,2016-01-24,2016,1,24,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,29
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18
8,2016-0012,2016-02-08,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22
9,2016-0013,2016-02-10,2016,2,10,PEATON,PEATON,MASCULINO,16


#### Columna VICTIMA
Reviso si las categorías presentes en esta columna corresponden a las informadas en el Diccionario de Datos

In [ ]:
df_HV['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL'], dtype=object)

Encuentro que todos corresponden, no hay ningún valor diferente a los especificados.

#### Verificacion de tipos de variables

In [ ]:
df_HV.dtypes

ID_hecho            object
FECHA       datetime64[ns]
AAAA                 int64
MM                   int64
DD                   int64
ROL                 object
VICTIMA             object
SEXO                object
EDAD                 int64
dtype: object

In [ ]:
df_HV['ID_hecho'] = df_HV['ID_hecho'].astype(str)
df_HV['ROL']=df_HV['ROL'].astype(str)
df_HV['VICTIMA']=df_HV['VICTIMA'].astype(str)
df_HV['SEXO']=df_HV['SEXO'].astype(str)

#### Cambios en los nombres de las columnas

In [ ]:
df_HV = df_HV.rename(columns={'ID_hecho': 'ID',
                                'FECHA': 'Fecha',
                                'AAAA':'Año',
                                'MM':'Mes',
                                'DD':'Día',
                                'ROL': 'Rol',                               
                                'VICTIMA': 'Víctima',
                                'SEXO': 'Sexo',
                                'EDAD':'Edad'                                
                                })

In [ ]:
df_HV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ID       717 non-null    object        
 1   Fecha    717 non-null    datetime64[ns]
 2   Año      717 non-null    int64         
 3   Mes      717 non-null    int64         
 4   Día      717 non-null    int64         
 5   Rol      717 non-null    object        
 6   Víctima  717 non-null    object        
 7   Sexo     717 non-null    object        
 8   EDAD     717 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 50.5+ KB


In [ ]:
#Exporto a CSV para poder continuar el trabajo luego
df_HV.to_csv('Homicidios_Victimas.csv')

### Preparación del dataset para proceso de EDA, KPI y construcción del DashBoard

Extraigo los datos ya limpios y transformados de los archivos CSV en los que se almacenaron anteriormente

In [ ]:
df_hoja_hechos = pd.read_csv('Homicidios_Hechos.csv', index_col=0)
df_hoja_victimas = pd.read_csv('Homicidios_Victimas.csv', index_col=0)

In [ ]:
df_hoja_hechos.head()

,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS


In [ ]:
df_hoja_hechos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   Cantidad víctimas      696 non-null    int64  
 2   Fecha                  696 non-null    object 
 3   Año                    696 non-null    int64  
 4   Mes                    696 non-null    int64  
 5   Día                    696 non-null    int64  
 6   Hora                   696 non-null    object 
 7   Hora entera            696 non-null    int64  
 8   Lugar del hecho        696 non-null    object 
 9   Tipo de Calle          696 non-null    object 
 10  Calle                  696 non-null    object 
 11  Cruce                  696 non-null    bool   
 12  Dirección Normalizada  696 non-null    object 
 13  Comuna                 696 non-null    int64  
 14  XY (CABA)              696 non-null    object 
 15  Pos x      

In [ ]:
df_hoja_victimas.head()

,ID,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,EDAD
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29


In [ ]:
df_hoja_victimas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       717 non-null    object
 1   Fecha    717 non-null    object
 2   Año      717 non-null    int64 
 3   Mes      717 non-null    int64 
 4   Día      717 non-null    int64 
 5   Rol      717 non-null    object
 6   Víctima  717 non-null    object
 7   Sexo     717 non-null    object
 8   EDAD     717 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 56.0+ KB


Como hay columnas repetidas, voy a eliminar algunas antes de mezclar los dos dataframes

In [ ]:
df_hoja_victimas=df_hoja_victimas.drop(columns=['Fecha', 'Año', 'Mes', 'Día'])

In [ ]:
df_hoja_hechos=df_hoja_hechos.drop(columns=['Víctima'])

Realizo un merge entre ambos dataframes

In [ ]:
df_homicidios = pd.merge(df_hoja_hechos, df_hoja_victimas, on='ID', how='inner')
df_homicidios

,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de Calle,...,Comuna,XY (CABA),Pos x,Pos y,Participantes,Acusado,Rol,Víctima,Sexo,EDAD
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,27
715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,9,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,60


In [ ]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     717 non-null    object 
 1   Cantidad víctimas      717 non-null    int64  
 2   Fecha                  717 non-null    object 
 3   Año                    717 non-null    int64  
 4   Mes                    717 non-null    int64  
 5   Día                    717 non-null    int64  
 6   Hora                   717 non-null    object 
 7   Hora entera            717 non-null    int64  
 8   Lugar del hecho        717 non-null    object 
 9   Tipo de Calle          717 non-null    object 
 10  Calle                  717 non-null    object 
 11  Cruce                  717 non-null    bool   
 12  Dirección Normalizada  717 non-null    object 
 13  Comuna                 717 non-null    int64  
 14  XY (CABA)              717 non-null    object 
 15  Pos x 

Hago una verificación extra de la columna víctima, para los casos que tuvieron varias víctimas fatales. Quiero confirmar que para estos casos aparezcan varios registros con el mismo ID, correspondientes a cada una de las víctimas, y que la información sí corresponde.

In [ ]:
df_homicidios[df_homicidios['Cantidad víctimas']>1]

,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de Calle,...,Comuna,XY (CABA),Pos x,Pos y,Participantes,Acusado,Rol,Víctima,Sexo,EDAD
29,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,...,7,Point (100232.38564985 99530.25843190),-58.460766,-34.633504,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,54
30,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,...,7,Point (100232.38564985 99530.25843190),-58.460766,-34.633504,MOTO-CARGAS,CARGAS,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,39
98,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,...,10,Point (94275.54271123 100886.87954649),-58.525721,-34.621259,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,37
99,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,...,10,Point (94275.54271123 100886.87954649),-58.525721,-34.621259,AUTO-CARGAS,CARGAS,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60
163,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,...,4,Point (104113.67806500 97722.68219304),-58.418428,-34.649791,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,FEMENINO,23
164,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,...,4,Point (104113.67806500 97722.68219304),-58.418428,-34.649791,AUTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,AUTO,MASCULINO,19
173,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,1,Point (109583.11620052 102006.72069921),-58.358815,-34.611136,AUTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,AUTO,MASCULINO,28
174,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,1,Point (109583.11620052 102006.72069921),-58.358815,-34.611136,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32
175,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,1,Point (109583.11620052 102006.72069921),-58.358815,-34.611136,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30
176,2017-0036,2,2017-03-29,2017,3,29,18:00:00,18,CURUPAYTI Y COLECTORA AV. GRAL. PAZ,GRAL PAZ,...,12,Point (95300.18060161 105287.23850698),-58.514523,-34.581598,MOTO-PASAJEROS,PASAJEROS,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,50


Este dataset está listo para pasar a la fase de EDA, elaborar los KPI's y llevar a un dashboard

In [ ]:
#Exporto a CSV para poder continuar el trabajo luego
df_homicidios.to_csv('homicidios_completo.csv')

## Base de datos Población CABA

En la página web del censo 2022 del Gobierno Argentino, encontré un documento en excel llamado Cuadro 1.1. Ciudad Autónoma de Buenos Aires. Total de población, variación absoluta y variación relativa, por comuna. Años 2010 y 2022. 

Creé una copia en excel de este documento y, utilizando la información de la variación absoluta y relativa, obtuve los valores proyectados para la población de los años 2017 a 2021. Este documento extendido es el que voy a utilizar a continuación en el proceso de calcular los KPIs

In [4]:
df_poblacion = pd.read_excel('Poblacion_CABA.xlsx')
df_poblacion

,Comuna,Población 2010,Población 2022,Variación absoluta,Variación relativa (%),Aumento población/año,Población 2016,Población 2017,Población 2018,Población 2019,Población 2020,Población 2021
0,CABA,2890151,3121707,231556,8.0,19296.333333,3005929.0,3.025225e+06,3.044522e+06,3063818.00,3.083114e+06,3.102411e+06
1,1,205886,223554,17668,8.6,1472.333333,214720.0,2.161923e+05,2.176647e+05,219137.00,2.206093e+05,2.220817e+05
2,2,157932,161645,3713,2.4,309.416667,159788.5,1.600979e+05,1.604073e+05,160716.75,1.610262e+05,1.613356e+05
3,3,187537,196240,8703,4.6,725.250000,191888.5,1.926138e+05,1.933390e+05,194064.25,1.947895e+05,1.955148e+05
4,4,218245,229240,10995,5.0,916.250000,223742.5,2.246588e+05,2.255750e+05,226491.25,2.274075e+05,2.283238e+05
5,5,179005,194271,15266,8.5,1272.166667,186638.0,1.879102e+05,1.891823e+05,190454.50,1.917267e+05,1.929988e+05
6,6,176076,203043,26967,15.3,2247.250000,189559.5,1.918068e+05,1.940540e+05,196301.25,1.985485e+05,2.007958e+05
7,7,220591,215896,-4695,-2.1,-391.250000,218243.5,2.178522e+05,2.174610e+05,217069.75,2.166785e+05,2.162872e+05
8,8,187237,204367,17130,9.1,1427.500000,195802.0,1.972295e+05,1.986570e+05,200084.50,2.015120e+05,2.029395e+05
9,9,161797,169063,7266,4.5,605.500000,165430.0,1.660355e+05,1.666410e+05,167246.50,1.678520e+05,1.684575e+05


In [5]:
df_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Comuna                  16 non-null     object 
 1   Población 2010          16 non-null     int64  
 2   Población 2022          16 non-null     int64  
 3   Variación absoluta      16 non-null     int64  
 4   Variación relativa (%)  16 non-null     float64
 5   Aumento población/año   16 non-null     float64
 6   Población 2016          16 non-null     float64
 7   Población 2017          16 non-null     float64
 8   Población 2018          16 non-null     float64
 9   Población 2019          16 non-null     float64
 10  Población 2020          16 non-null     float64
 11  Población 2021          16 non-null     float64
dtypes: float64(8), int64(3), object(1)
memory usage: 1.6+ KB


# KPI'S

Se nos ha pedido calcular los siguientes KPI:
1. Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

2. Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100


A partir de la base de datos de población y la de homicidios voy a extraer la información necesaria para hacer los calculos correspondientes

In [6]:
df_homicidios= pd.read_csv('homicidios_completo.csv')
df_homicidios

,Unnamed: 0,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,...,Comuna,XY (CABA),Pos x,Pos y,Participantes,Acusado,Rol,Víctima,Sexo,EDAD
0,0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19
1,1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70
2,2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30
3,3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18
4,4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,...,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
713,713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,...,9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
714,714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,...,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,27
715,715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,...,9,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,60


In [7]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             717 non-null    int64  
 1   ID                     717 non-null    object 
 2   Cantidad víctimas      717 non-null    int64  
 3   Fecha                  717 non-null    object 
 4   Año                    717 non-null    int64  
 5   Mes                    717 non-null    int64  
 6   Día                    717 non-null    int64  
 7   Hora                   717 non-null    object 
 8   Hora entera            717 non-null    int64  
 9   Lugar del hecho        717 non-null    object 
 10  Tipo de Calle          717 non-null    object 
 11  Calle                  717 non-null    object 
 12  Cruce                  717 non-null    bool   
 13  Dirección Normalizada  717 non-null    object 
 14  Comuna                 717 non-null    int64  
 15  XY (CA

#### Para calcular el primer KPI necesito obtener la cantidad de muertes correspondientes a cada semestre de los años registrados, del dataframe df_homicidios

In [8]:
#Creo una columna semestre para hacer más fácil el cálculo
def determinar_semestre(mes):
    if 1 <= mes <= 6:
        return 1
    elif 7 <= mes <= 12:
        return 2
    else:
        return 'Mes no válido'

# Aplicar la función a la columna 'meses' y crear una nueva columna 'semestre'
df_homicidios['Semestre'] = df_homicidios['Mes'].apply(determinar_semestre)

In [9]:
df_homicidios

,Unnamed: 0,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,...,XY (CABA),Pos x,Pos y,Participantes,Acusado,Rol,Víctima,Sexo,EDAD,Semestre
0,0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,1
1,1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,...,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,1
2,2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,...,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,1
3,3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,...,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,1
4,4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,...,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,...,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2
713,713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,...,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2
714,714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,...,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,27,2
715,715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,...,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,60,2


In [10]:
#agrupo por año y por semestre para determinar la cantidad de víctimas

df_victimas_por_semestre= df_homicidios.groupby(['Año', 'Semestre'])['Cantidad víctimas'].count().reset_index()
df_victimas_por_semestre

,Año,Semestre,Cantidad víctimas
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70
5,2018,2,79
6,2019,1,57
7,2019,2,47
8,2020,1,31
9,2020,2,50


In [11]:
#me aseguro de que el tipo de dato de la columna Año sea entero para poder hacer el merge sin problema
df_victimas_por_semestre['Año'] = df_victimas_por_semestre['Año'].astype(int)

In [12]:
df_victimas_por_semestre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Año                12 non-null     int64
 1   Semestre           12 non-null     int64
 2   Cantidad víctimas  12 non-null     int64
dtypes: int64(3)
memory usage: 420.0 bytes


Ahora debo extraer la información de la población total de CABA de cada año, del dataframe df_poblacion

In [13]:
#Extraigo la información de población para cada año
df_poblacion_total= df_poblacion[['Población 2016','Población 2017', 'Población 2018','Población 2019', 
                                  'Población 2020', 'Población 2021']]

df_poblacion_total

#cambio nombres de las columnas
df_poblacion_total = df_poblacion_total.rename(columns={'Población 2016':'2016',
                                                        'Población 2017': '2017', 
                                                        'Población 2018': '2018',
                                                        'Población 2019': '2019', 
                                                        'Población 2020': '2020', 
                                                        'Población 2021': '2021'                  
                                                        })




In [14]:
#Me quedo sólo con los valores de la primera fila que son los valores totales
df_poblacion_total = df_poblacion_total[:1]
df_poblacion_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2016    1 non-null      float64
 1   2017    1 non-null      float64
 2   2018    1 non-null      float64
 3   2019    1 non-null      float64
 4   2020    1 non-null      float64
 5   2021    1 non-null      float64
dtypes: float64(6)
memory usage: 180.0 bytes


In [15]:
#modifico la orientacíon del dataframe para que quede en el formato de filas y columnas que necesito
df_poblacion_anios = pd.melt(df_poblacion_total, var_name='Año', value_name='Población Total').sort_values(by='Año')

In [16]:
df_poblacion_anios

,Año,Población Total
0,2016,3.005929e+06
1,2017,3.025225e+06
2,2018,3.044522e+06
3,2019,3.063818e+06
4,2020,3.083114e+06
5,2021,3.102411e+06


In [17]:
#me aseguro de que el tipo de dato de la columna Año sea entero para poder hacer el merge sin problema
df_poblacion_anios['Año'] = df_poblacion_anios['Año'].astype(int)

In [18]:
df_poblacion_anios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Año              6 non-null      int64  
 1   Población Total  6 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 228.0 bytes


In [19]:
#uno ambas tablas
df_kpi1 = pd.merge(df_victimas_por_semestre, df_poblacion_anios, on='Año', how='left')
df_kpi1

,Año,Semestre,Cantidad víctimas,Población Total
0,2016,1,65,3.005929e+06
1,2016,2,81,3.005929e+06
2,2017,1,69,3.025225e+06
3,2017,2,71,3.025225e+06
4,2018,1,70,3.044522e+06
5,2018,2,79,3.044522e+06
6,2019,1,57,3.063818e+06
7,2019,2,47,3.063818e+06
8,2020,1,31,3.083114e+06
9,2020,2,50,3.083114e+06


In [20]:
#añado una columna con la tasa de homicidios para cada semestre
df_kpi1['Tasa semestral'] = df_kpi1['Cantidad víctimas']*100000/df_kpi1['Población Total']
df_kpi1
    

,Año,Semestre,Cantidad víctimas,Población Total,Tasa semestral
0,2016,1,65,3.005929e+06,2.162393
1,2016,2,81,3.005929e+06,2.694674
2,2017,1,69,3.025225e+06,2.280822
3,2017,2,71,3.025225e+06,2.346933
4,2018,1,70,3.044522e+06,2.299212
5,2018,2,79,3.044522e+06,2.594825
6,2019,1,57,3.063818e+06,1.860424
7,2019,2,47,3.063818e+06,1.534034
8,2020,1,31,3.083114e+06,1.005477
9,2020,2,50,3.083114e+06,1.621737


In [21]:
#exporto este dataframe a csv para ser utilizado en Tabelau para elaborar el dashboard
df_kpi1.to_csv('data_kpi01.csv')

#### Para calcular el segundo KPI necesito obtener la cantidad de muertes de motociclistas ocurridas, en cada año, del dataframe df_homicidios

In [22]:
#selecciono todos los registros en los que la víctima iba en MOTO
df_motociclistas = df_homicidios[df_homicidios['Víctima']=='MOTO']
df_motociclistas

,Unnamed: 0,ID,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,...,XY (CABA),Pos x,Pos y,Participantes,Acusado,Rol,Víctima,Sexo,EDAD,Semestre
0,0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,1
2,2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,...,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,1
3,3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,...,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,1
4,4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,...,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,1
5,5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,...,Point (101721.59002217 93844.25656649),-58.444513,-34.684759,MOTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,MOTO,MASCULINO,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,708,2021-0089,1,2021-12-02,2021,12,2,01:10:00,1,AV. GAONA 3655,...,Point (98804.41713890 100872.30706871),-58.476337,-34.621406,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,41,2
710,710,2021-0091,1,2021-12-11,2021,12,11,23:00:00,23,BAIGORRIA Y VICTOR HUGO,...,Point (94810.03686085 100710.80080255),-58.519894,-34.622849,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,24,2
712,712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,...,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2
713,713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,...,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2


In [23]:
#agrupo por año y obtengo total de accidentes en motos anuales
df_accidentes_motos = df_motociclistas.groupby('Año')['Víctima'].count().reset_index()
df_accidentes_motos

,Año,Víctima
0,2016,65
1,2017,56
2,2018,57
3,2019,50
4,2020,29
5,2021,46


In [24]:
#me aseguro de que el tipo de dato de la columna Año sea entero para poder hacer el merge sin problema
df_accidentes_motos['Año'] = df_accidentes_motos['Año'].astype(int)

In [25]:
#uno esta tabla con la de población total por años
df_kpi2 = pd.merge(df_accidentes_motos, df_poblacion_anios, on='Año', how='left')
df_kpi2


,Año,Víctima,Población Total
0,2016,65,3.005929e+06
1,2017,56,3.025225e+06
2,2018,57,3.044522e+06
3,2019,50,3.063818e+06
4,2020,29,3.083114e+06
5,2021,46,3.102411e+06


In [26]:
#añado una columna con el número de víctimas del periodo anterior
df_kpi2["Víctimas año anterior"] = df_kpi2["Víctima"].shift(periods=1, fill_value=0)
df_kpi2


,Año,Víctima,Población Total,Víctimas año anterior
0,2016,65,3.005929e+06,0
1,2017,56,3.025225e+06,65
2,2018,57,3.044522e+06,56
3,2019,50,3.063818e+06,57
4,2020,29,3.083114e+06,50
5,2021,46,3.102411e+06,29


In [27]:
#finalmente calculo el valor que me va a indicar la evolución en el número de accidentes por año
df_kpi2['Evolución']= (df_kpi2['Víctimas año anterior']-df_kpi2['Víctima'])*100/df_kpi2['Víctimas año anterior']
df_kpi2

,Año,Víctima,Población Total,Víctimas año anterior,Evolución
0,2016,65,3.005929e+06,0,-inf
1,2017,56,3.025225e+06,65,13.846154
2,2018,57,3.044522e+06,56,-1.785714
3,2019,50,3.063818e+06,57,12.280702
4,2020,29,3.083114e+06,50,42.000000
5,2021,46,3.102411e+06,29,-58.620690


In [28]:
#Cambio nombres de las columnas
df_kpi2 = df_kpi2.rename(columns={'Víctima':'Cantidad de víctimas' })

In [31]:
df_kpi2.to_csv('data_kpi02.csv')